In [1]:
import pandas as pd
import numpy as np
import re

def load_dataframe(relative_path,dataframe_name):
    df = pd.read_pickle(f'{relative_path}/{dataframe_name}.pkl')    
    return df

def read_file(relative_path,file_name):
    text= ""
    with open(f'{relative_path}/{file_name}.abc','r') as f:
        text = f.read()
    return text

In [2]:
relative_path ="notebooks/data/final_dataset"
filename_name = 'clean_augmented_data'
#filename_name = 'clean_original_training_data'
#relative_path ="notebooks/data/original_dataset"
training_data_df = load_dataframe(relative_path,filename_name)
training_data_df.columns


Index(['unit_note_length', 'tuneBook', 'title', 'reference_number',
       'original_header', 'original_body', 'meter', 'key', 'clean_song',
       'clean_header', 'clean_body', 'chord_progression', '"fm"', '"ff'"',
       '"f7"', '"em"', '"ee'"', '"e7"', '"e"', '"dm"', '"dd'"', '"d7"', '"d"',
       '"cm"', '"cc'"', '"c7"', '"c#m"', '"c#7"', '"c"', '"Gm"', '"Gg"',
       '"Gd'"', '"G7"', '"G#m"', '"G#7"', '"G"', '"Fm"', '"Ff"', '"Fc'"',
       '"F7"', '"F#m"', '"F#7"', '"F"', '"Em"', '"Eb"', '"E7"', '"E#m"',
       '"E#7"', '"E"', '"Dm"', '"Da"', '"D7"', '"D#m"', '"D#7"', '"D"', '"Cm"',
       '"Cg"', '"C7"', '"C#m"', '"C#7"', '"C"', '"Bm"', '"Bf"', '"Bb"', '"B7"',
       '"B#m"', '"B#7"', '"B"', '"Am"', '"Ae'"', '"Aa"', '"A7"', '"A#m"',
       '"A#7"', '"A"'],
      dtype='object')

In [3]:
training_data_df.tail()

,unit_note_length,tuneBook,title,reference_number,original_header,original_body,meter,key,clean_song,clean_header,...,"""B#m""","""B#7""","""B""","""Am""","""Ae'""","""Aa""","""A7""","""A#m""","""A#7""","""A"""
9491,1/4,dataset_min5.abc,Grandpa's,78,X:78\nT:Grandpa's\nM:4/4\nL:1/4\nK:Amajor,"E/2D/2|""A,""CE""E7""FG|""A,""A/2G/2A/2B/2ce|""B,m""dc...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""|""Bm""""B7""|""E7""|...","M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""|""Bm""""B7""|""E7""|...",...,0,0,0,0,0,0,0,0,0,9
9492,1/4,dataset_min5.abc,The Girl With The Green Hat On,79,X:79\nT:The Girl With The Green Hat On\nM:4/4\...,"(3E/2F/2G/2|""A,""AE""E7""E/2F/2E/2D/2|""A,""C/2D/2E...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""""E7""|""A""|""E7""|""...","M:4/4\nL:1/4\nK:A\n|""A""""E7""|""A""""E7""|""A""|""E7""|""...",...,0,0,0,0,0,0,0,0,0,9
9493,1/4,dataset_min5.abc,Green Meadow,80,X:80\nT:Green Meadow\nM:4/4\nL:1/4\nK:Dmajor,"(3A,/2B,/2C/2|""D""DD/2E/2F/2D/2F/2A/2|""G,""B/2c/...",4/4,D,"M:4/4\nL:1/4\nK:D\n|""D""|""G""""D""|""Em""|""Em""""A7""|""...","M:4/4\nL:1/4\nK:D\n|""D""|""G""""D""|""Em""|""Em""""A7""|""...",...,0,0,0,0,0,0,7,0,0,0
9494,1/4,dataset_min5.abc,The Old Grey Cat,82,X:82\nT:The Old Grey Cat\nM:4/4\nL:1/4\nK:Bminor,"F|""B,m""BBB,B,/2C/2|""B,m""D/2C/2D/2E/2F/2E/2F/2^...",4/4,Bm,"M:4/4\nL:1/4\nK:Bm\n|""Bm""|""Bm""|""A""|""A""|""Bm""|""B...","M:4/4\nL:1/4\nK:Bm\n|""Bm""|""Bm""|""A""|""A""|""Bm""|""B...",...,0,0,0,0,0,0,0,0,0,5
9495,1/4,dataset_min5.abc,Gyre And Gimble,84,X:84\nT:Gyre And Gimble\nM:4/4\nL:1/4\nK:Amajor,"E|""A,""AECE|""B,m""FD""E7""B,D|""A,""CEA3/2B/2|""E7""c/...",4/4,A,"M:4/4\nL:1/4\nK:A\n|""A""|""Bm""""E7""|""A""|""E7""""A""|""...","M:4/4\nL:1/4\nK:A\n|""A""|""Bm""""E7""|""A""|""E7""""A""|""...",...,0,0,0,0,0,0,0,0,0,12


In [4]:
training_data_df["clean_header"].str.len().max()

373

In [5]:
training_data_df["clean_body"].str.len().max()

1257

In [6]:
bodies = ""
silences = 0
for body in training_data_df["clean_body"]:
    if 'z' in body:
        silences +=1 
    bodies += body+"\n"
chars = sorted(list(set(bodies)))
vocab_size = len(chars)
print('vocab: ',''.join(chars))
print('vocab_size',vocab_size)
print("silences ",silences)

vocab:  
"#'(),-/1234567=ABCDEFG[]^_abcdefgmz|~
vocab_size 39
silences  516


In [7]:
training_data_text = read_file(relative_path,filename_name)

print("number of chars:",len(training_data_text))

number of chars: 4062773


In [7]:
chars = sorted(list(set(training_data_text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

NameError: name 'training_data_text' is not defined

In [9]:
import torch

print(torch.__version__)

2.0.0+cu117


In [10]:
import transformers
print(transformers.__version__)

4.28.0.dev0


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
import wandb
import tiktoken

print(wandb.__version__)

0.14.2


In [2]:
!ls

Dockerfile  docker-compose.yaml  overrides.json
README.md   notebooks		 requirements.txt


In [3]:
from dotenv import load_dotenv
import os
nano_path = 'notebooks/nanoGPT'
os.chdir(nano_path)
load_dotenv()

True

In [8]:
!ls

LICENSE      config	      out-abc-char	    transformer_sizing.ipynb
README.md    configurator.py  out-shakespeare-char  wandb
__pycache__  data	      sample.py
assets	     model.py	      scaling_laws.ipynb
bench.py     older_ckpt       train.py


In [ ]:
# Dataset with multiple voices present
#length of dataset in characters: 4,149,703
#all the unique characters: 
#"#'()+,-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
#vocab size: 46
#train has 3,734,732 tokens
#val has 414,971 tokens

In [14]:
!python3 data/abc_char/prepare.py

length of dataset in characters: 4,062,773
all the unique characters: 
"#'(),-/123456789:=ABCDEFGKLM[]^_abcdefgmz|~
vocab size: 45
train has 3,656,495 tokens
val has 406,278 tokens


In [15]:
!python3 train.py config/train_abc_char.py

Overriding config with config/train_abc_char.py:
# train a miniature character-level shakespeare model
# good for debugging and playing on macbooks and such

out_dir = 'out-abc-char'
eval_interval = 10 # keep frequent because we'll overfit
eval_iters = 500
log_interval = 5 # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = True # override via command line if you like
wandb_project = 'abc-char'
wandb_run_name = 'mini-char-gpt'

dataset = 'abc_char'
batch_size = 32
block_size = 512 # context of up to 512 previous characters

# baby GPT model :)
n_layer = 12
n_head = 12
n_embd = 384
dropout = 0.2

learning_rate = 1e-3 # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000 # make equal to max_iters usually
min_lr = 1e-4 # learning_rate / 10 usually
beta2 = 0.99 # make a bit bigger because number of tokens per iter is small

warmup_iters = 5 # not super neces

## Test key with most occurrences: G

In [16]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]'

Overriding: out_dir = out-abc-char
Overriding: start = M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
number of parameters: 21.26M
abc_char
Loading meta from data/abc_char/meta.pkl...
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
Bc|"G"d2dB|dBGB|"C"cBAG|E2E2|"C"e2ee|ecAG|"D"FAAB|A2D2|"D"DFAB|dAFA|"D"d4-|"G"d4|d4|"C"efee|ecAG|E4|"D"D2F2|"G"B2d2|d2B|"C"cBAG|E2E2|"D"FABc|"D"d2d2|"G"d2d2|"C"efed|"D"d4|"G"B4|"C"c4|]

M:4/4
L:1/4
K:G
|"G"|"G""C"|"G"|"D"|"Em""D"|"D"|"G""C"|"G"|"G""C"|"G""C"|"G""D"|"Em""D"|"D"|"Em""D"|"Em""D"|]
D|"G"GB/2G/2B/2G/2B/2G/2|"G"d/2B/2"C"e/2c/2"G"B/2G/2B/2G/2|"D"Ac/2A/2F/2A/2c/2|"Em"BB/2G/2"D"A/2F/2D/2F/2|"D"D/2E/2F/2G/2A/2B/2c/2|"G"d/2B/2"C"e/2c/2"G"B/2G/2B/2d/2|"G"d"C"e/2c/2"G"B/2G/2B/2d/2|"D"aa/2f/
---------------
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
Bc|"G"dBd^c/2d/2|d/2B/2GG^^G|"C"A2EF|A/2E/2G/2A/2GE|"D"D=^CB,B,|D/2E/2DB,=B,|"D"A,B,^B,=B,|"D"A,4|"D"A/2^^C/2AA|A/2^G/2A/2G/2A/2B/2A/2|g/2A/2G/2A/2BA/2G/2|"G"B/2^A/2GG|"C"A/2^^G/2A/2G/2E/2D/2E/2|A/2E/2G/

## Test major key with low samples: C

In [17]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]'

Overriding: out_dir = out-abc-char
Overriding: start = M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]
number of parameters: 21.26M
abc_char
Loading meta from data/abc_char/meta.pkl...
M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]
EF|"C"CE/2G/2cB/2A/2G/2|"F"AFF/2A/2F/2A/2|"G"Ggg^f|"G"d/2^c/2d/2e/2d/2c/2B/2G/2|"C"CE"CEF/2A/2|"C"Ggec/2G/2c/2E/2|"F"CDFA|"G"GgGf|"G"d/2^c/2d/2e/2d/2c/2B/2A/2G/2|"C"CEFG|"F"AFF|"C"Egg^f/2g/2|"F"a/2g/2e/2f/2d/2c/2B/2c/2|"G"d/2^c/2d/2e/2d/2c/2B/2G/2|"C"Cgfe/2c/2|"F"Aff^f|"G"d/2^c/2d/2e/2d/2c/2B/2c/2|"C"Egg^f|"F"AFF|"C"Egg^^F/2G/2|"F"Acf^f|"G"d/2^c/2d/2e/2d/2c/2B/2c/2|"C"Egg^f|"F"AFFe/2d/2|"G"GGf^^F|"C"Egg^^F/2G/2|"F"Aff^^F|"Bb"GDFA|"C"Ggg^^F/2G/2A/2B/2|"F"cFF|]

M:4/4
L:1/4
K:G
|"Gg"|"Cg"|"Gg""D7
---------------
M:4/4L:1/4K:C|"C"|"F"|"G"|"G"|"C"|"F"|"G"|"C"|]
G|"C"e/2d/4c/4B/2G/2|"F"A/2c/2c/2A/2|A/2c/2G/2F/2|"F"A/2c/2d/2c/2|A/2c/2G/2F/2|"G"B/2B/2B/2A/2G/4A/4|"C"G/2c/2c/2G/2|E/2G/2C/2c/2|"F"A/2c/2c/2d/2c/2|A/2c/2G/2F/2|"G"G/2B/2B/2D/2G/2|"C"E/2G/2C|]

M:4/4
L:

## Test minor key with low samples: Am

In [18]:
!python3 sample.py --out_dir=out-abc-char --start='M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]'

Overriding: out_dir = out-abc-char
Overriding: start = M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]
number of parameters: 21.26M
abc_char
Loading meta from data/abc_char/meta.pkl...
M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]
E|"Am"C/2A/2B/4A/4G/2|A/2c/2c/4B/4A/2|"Am"C/2A/2B/4A/2G/2|"Dm"F/2A/2G/2F/2|"E"EE|"Am"A/2c/4B/4A/2G/2|A/2c/2c/4B/4A/2|"Dm"F/2A/2G/2F/2|D/2F/2A/4G/4|"E"E/2E/2E"Am"A/4B/4c/4B/2A/2|"Am"A/2c/2c/4B/4A/2|"Dm"F/2A/2G/2F/2|D/2F/2A/4G/4F/2|"E"E/2E/2E|]

M:4/4
L:1/4
K:C
|"C""G7"|"C""G7"|"C"|"Am""G7"|"C""G7"|"C""G7"|"C"|"G"|"F""Am"|"G7"|"C""D7"|"G"|"F""G7"|"C""G7"|"C"|"G"|"F""Am"|"E#m""Am"|"D7""G7"|"C"|"G"|]
G/2A/2|"C"cc"G7"Bc/2d/2|"C"ee"G7"dc/2B/2|"C"cc/2A/2G/2E/2G/2|"Am"cc"G7"Bc/2d/2|"C"ee/2f/2eg|"Am"a/2
---------------
M:4/4L:1/4K:Am|"Am"|"Dm"|"E"|"E"|"Am"|"Dm"|"E"|"Am"|]
"Am"ae/2d/2c/2B/2A/2|"Dm"d/2^c/2d/2e/2f/2e/2d/2c/2|"E"^B/2E/2EE/2F/2G/2E/2|"Am"c/2B/2A/2G/2A/2B/2c/2d/2|"E"^B/2E/2EE/2^F/2G/2|"Am"A/2^G/2A/2B/2c/2d/2e/2^f/2|"Dm"g/2f/2e/2c/2"Dm"d/2^c/2d

## Test older checkpoint

In [24]:
 !python3 sample.py --out_dir=older_ckpt/m_voices --path_meta=older_ckpt/m_voices --start='M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]'

Overriding: out_dir = older_ckpt/m_voices
Overriding: path_meta = older_ckpt/m_voices
Overriding: start = M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
number of parameters: 14.18M
shakespeare_char
Loading meta from older_ckpt/m_voices/meta.pkl...
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
e/2|"G"gg/2a/2gd/2c/2|"G"B/2G/2A/2G/2[+GB][+GB][+GB]|"D"A/2G/2F/2E/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][+GB]e/2f/2|"G"gg/2a/2g/2e/2d/2e/2|g/2f/2g/2a/2bb/2e/2|"D"f/2e/2f/2g/2a/2f/2a/2f/2|"G"g/2f/2g/2a/2"C"g/2e/2d/2e/2|"G"gg/2a/2g/2e/2d/2e/2|g/2f/2g/2a/2bb|"D"d'd'"D7"e/2f3/2|"G"g3/2a/2gG/2E/2|"G"D/2B,/2D/2G/2D/2B,/2D/2G/2|"C"E/2[CE][C/2E/2][CE][CE]|"D"A/2B/2A/2F/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][GB]G/2E/2|"G"D/2B,/2D/2G/2D/2B,/2D/2G/2|"C"E/2[CE][C/2E/2][CE][CE]|"D"A/2B/2A/2F/2D/2F/2A/2c/2|"G"B/2[GB][G
---------------
M:4/4L:1/4K:G|"G"|"C"|"D"|"D"|"G"|"C"|"D"|"G"|]
e/2|"G"gg/2a/2gd/2c/2|"G"B/2G/2A/2G/2[+GB][+GB]|"D"A/2G/2F/2E/2D/2F/2A/2c/2|"G"B/2[GB][G/2B/2][+GB]e/2f/2|"G"gg/2a/2g/2e/2d/2c/2|"G"B/2G/2

In [ ]:
cat older_ckpt/m_voices/ckpt.pkl

In [ ]:
!ls -l older_ckpt/m_voices

In [ ]:
!ls -l out-abc-char/

In [ ]:
!id